In [30]:
#working
import requests
import csv
import re
import urllib.request
from bs4 import BeautifulSoup
import random,time,datetime

UDcount=0
print(datetime.datetime.now())
user_agent_list = []
useragent=open('Useragent.txt','r')
for ug in useragent:
    user_agent_list.append(ug.strip())
users=[]
sc=[]
f=open('instashortcode1.csv','r',encoding='utf-8')
for line in f:
    sc.append(line.strip())
for i in range (0,len(sc)):
    user_agent=user_agent_list[random.randint(0,4656)]
    row=[]
    if i%5==0:
        user_agent=user_agent_list[random.randint(0,4656)]
    try:   
        r = requests.get("https://www.instagram.com/p/"+str(sc[i]),headers={'User-Agent':user_agent })
    except OSError:
        print('Error at',i)
        user_agent=user_agent_list[random.randint(0,4656)]
        r = requests.get("https://www.instagram.com/p/"+sc[i],headers={'User-Agent':user_agent })
    if r.status_code is 200 :
        soup = BeautifulSoup(r.text)
    else:
        continue
    text2=[]
    data = soup.find_all('link', attrs={'rel': 'canonical'})
    text = data[0].get('href').split('/')
    if text[3]=='p':
        data2 = soup.find_all('meta', attrs={'property': 'og:description'})
        text2 = data2[0].get('content').split()
        for k in range(0,len(text2)):
            if '(@' in text2[k]:
                userid = text2[k].replace('(','').replace('@','').replace(')','')
                break
    else:
        userid=text[3]
    users.append(userid)
    print(i+1,sc[i],userid)
print('Taken Shortcodes:',len(sc))
print('Valid Shortcodes:',len(users))
print('Broken Shortcodes:',(len(sc)-len(users)))
users=list(set(users))
print('Unique Shortcodes:',len(users))
print('-----------------------------------------------------------')

#userids to userdetails
for i in range (0,len(users)):
    user_agent=user_agent_list[random.randint(0,4656)]
    row=[]
    if i%10==0:
        user_agent=user_agent_list[random.randint(0,4656)]
    try:
        r = requests.get("https://www.instagram.com/"+users[i],headers={'User-Agent':user_agent })
    except OSError:
        print('Error at',i)
        user_agent=user_agent_list[random.randint(0,4656)]
        r = requests.get("https://www.instagram.com/"+users[i],headers={'User-Agent':user_agent })
    if r.status_code is 200:
         soup = BeautifulSoup(r.text)
    else:
        continue
    data = soup.find_all('meta', attrs={'property': 'og:description'})
    text=[]
    text = data[0].get('content').split()
    
    #if 'k' not in text[0]:
    #    text[0] = int(text[0].replace(',',''))
    #    if text[0]>=1000:
    #        followers = text[0]     #for checking the followers 
    #    else:
    #        continue
    #else:
    #    followers = text[0]
    
    followers = text[0]
    posts = text[4]
    following = text[2]
    user_id=users[i]
    
        
    if 'k' in text[0]:         
        a=text[0].strip('k')    
        a=(float(a))*1000       
        a=int(a)
        followers=a               #for converting k's or m's in followers to normal integer
    elif 'm' in text[0]:
        a=text[0].strip('m ')
        a=(float(a))*1000000
        a=int(a)
        followers=a
    
    hol=z=[]
    emaildata = soup.find_all('script', attrs={'type': 'application/ld+json'})
    hol=str(emaildata)
    email=re.findall(r'\"(.+?)\"',hol)
    if 'alternateName' in email:
        text1=[]   
        l=text.index('from')
        if len(text)==l+2:
            user_name=users[i]
        else:
            for k in range(l+1,(len(text))-1):
                text1.append(text[k])      
                user_name=' '.join(text1)
    else:
        user_name='NA'
                               
    if "email" in email:
        email_id=email[email.index('email')+1]
    else:
        email_id='NA'
    
    if "description" in email:
        description=email[email.index('description')+1].replace("\n","")
    else:
        description='NA'
        
    if "telephone" in email:
        telephone=email[email.index('telephone')+1]
    else:
        telephone='NA'
        
    if "external_url" in email:
        external_url=email[email.index('external_url')+1]
    else:
        external_url='NA'
        
    emaildata=soup.find_all('script', attrs={'type': 'text/javascript'})
    g=x=str(emaildata[3])
    h=re.findall(r'[#]\w+',g)
    hashes=' '.join(h)
    business=re.findall(r'\"(.+?)\"',x)
   
    
    if "city_name\\" in business:
        cityname=business[business.index('city_name\\')+1]
        
        if cityname=='\\':
            cityname='NA'
        else:
            cityname=business[business.index('city_name\\')+1].replace('\\','')
        
    
    if "business_email" in business:
        b_email=business[business.index('business_email')+1]
        if "business" in b_email:
            b_email  = 'NA'
        elif "," in b_email:
            b_email = 'NA'
    else:
        b_email = 'NA'

    if "business_phone_number" in business:
        b_phoneno=business[business.index('business_phone_number')+1]
        if "business" in b_phoneno:
            b_phoneno  = 'NA'
        elif "," in b_phoneno:
            b_phoneno = 'NA'
    else:
        b_phoneno = 'NA'
    print(i)
    #row = [user_id,posts,followers,following,user_name,description,email_id,telephone,external_url,b_email,b_phoneno,hashes]#to print in csv
    row=[user_id,posts,followers,following,user_name,email_id,telephone,external_url,b_email,b_phoneno,cityname]#hashes
    print(row)
    with open('userdetails1.csv', 'a',encoding='utf-8') as csvFile:
        UDcount+=1
        writer = csv.writer(csvFile)
        writer.writerow(row)
    csvFile.close()
print('Taken Userids:',len(users))
print('Valid Userids:',UDcount)
print('Broken Userids',(len(users)-UDcount))
print(datetime.datetime.now())

2019-02-18 18:39:17.780434
1 BexmI52lpyN tanvimehtasrivastava
2 BexPNsZBMOF littlefoodiegirl_
3 BuA5PccjkW1 thatsassylady
Taken Shortcodes: 3
Valid Shortcodes: 3
Broken Shortcodes: 0
Unique Shortcodes: 3
-----------------------------------------------------------
0
['thatsassylady', '943', '7,127', '902', 'Samixa Tiwari', 'samixa20@gmail.com', 'NA', 'NA', 'samixa20@gmail.com', 'NA', 'Bangalore, India']
1
['tanvimehtasrivastava', '572', 14800, '1,990', 'PINK CRAYON', 'pinkcrayonmail@gmail.com', 'NA', 'NA', 'pinkcrayonmail@gmail.com', 'NA', 'NA']
2
['littlefoodiegirl_', '344', '3,424', '1,585', 'Vedshree Gupta', 'littlefoodiegirl@gmail.com', 'NA', 'NA', 'littlefoodiegirl@gmail.com', 'NA', 'NA']
Taken Userids: 3
Valid Userids: 3
Broken Userids 0
2019-02-18 18:39:27.663525


In [27]:
print(users)

[]


In [4]:
b=4
c=3
print('hello',b-c)

hello 1
